# Save and Load models in Pytorch

Here we use a simple CNN trained on the MNIST dataset to save and load its model

# Imports

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Model

In [5]:
# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Hyperparameter
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 10 
load_model = False

# Create the model
class CNN(nn.Module):
  def __init__(self, in_channels = 1, num_classes = 10):  # 28x28 = 784
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=(3,3), stride=(1,1), padding=(1,1)) # 28x28
    self.pool = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2)) # 14x14
    self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=(3,3), stride=(1,1), padding=(1,1))
    self.fc1 = nn.Linear(16*7*7, num_classes)

  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)
    return x

#  Define function to save checkpoint
def save_checkpoint(state, filename="mycheckpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state,filename)
    
def load_checkpoint(checkpoint):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

cuda


$$n_{out}=
\lfloor
\frac{n_{in}+2p-k}{s}+1
\rfloor
=
\lfloor
\frac{28+2-3}{1}+1
\rfloor
=
28
$$
where:

$n_{in}$ : number of input features

$n_{out}$ : number of output features

$k$ : convolution kernel size

$p$ : convolution padding size

$s$ : convolution stride size

# Load Data

In [6]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Training

In [7]:
# Intialize NN
model = CNN().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

if load_model:
    load_checkpoint(torch.load("mycheckpoint.pth.tar"))

# Train the NN
for epoch in range(num_epochs):
    # save checkpoint
    if epoch % 3 == 0:
        checkpoint = {"state_dict" : model.state_dict(),
                     "optimizer" : optimizer.state_dict()}
        save_checkpoint(checkpoint)
    
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to device
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Forward propagation
        scores = model(data)
        loss = criterion(scores, targets) 
        
        # Backward propagation
        optimizer.zero_grad() # initialize all gradients to zero for each batch
        loss.backward()

        # Gradient descent or Adam step
        optimizer.step()
    
    # Print loss of epoch
    print(f"Loss at epoch {epoch} was {loss:.05f}")

=> Saving checkpoint
Loss at epoch 0 was 0.02654
Loss at epoch 1 was 0.27576
Loss at epoch 2 was 0.05173
=> Saving checkpoint
Loss at epoch 3 was 0.05511
Loss at epoch 4 was 0.01024
Loss at epoch 5 was 0.01146
=> Saving checkpoint
Loss at epoch 6 was 0.09222
Loss at epoch 7 was 0.01471
Loss at epoch 8 was 0.00268
=> Saving checkpoint
Loss at epoch 9 was 0.00082


# Performance

In [8]:
# Check accuracy on training and test sets
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)  # scores is 64x10 and we want to know which one of those the is the maximum value, so in max: dim=1
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    print(f'got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()

In [9]:
load_checkpoint(torch.load("mycheckpoint.pth.tar"))

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

=> Loading checkpoint
Checking accuracy on training data
got 59425 / 60000 with accuracy 99.04
Checking accuracy on test data
got 9869 / 10000 with accuracy 98.69
